In [1]:
import os, sys
import importlib
import matplotlib.pyplot as plt 
import numpy as np
sys.path.append("../../../../src")
sys.path.append("../../../../../hi-ml/src")
sys.path.append("../../../../../hi-ml-azure/src")
import histopathology.configs.classification.DeepSMILEPanda as mil_configs
import histopathology.models.deepmil as deepmil
import matplotlib.pyplot as plt

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cloudpickle 2.0.0 (/home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages), Requirement.parse('cloudpickle<2.0.0,>=1.1.0'), {'azureml-dataprep'}).


In [2]:
data_root = "/tmp/datasets"
panda_tiles_root = os.path.join(data_root, "PANDA_tiles")
panda_wsi_root = os.path.join(data_root, "PANDA")
root = "~/workspace/repos/hi-ml/hi-ml-histopathology/src/histopathology/configs/classification"
train_csv = os.path.join(root, "custom_subset/panda/sub_train_tiles.csv")
val_csv = os.path.join(root, "custom_subset/panda/sub_val_tiles.csv")

# train_csv = os.path.join(root, "custom_subset/panda/sub_train_slides.csv")
# val_csv = os.path.join(root, "custom_subset/panda/sub_val_slides.csv")

## Tiles Exploration

### MIL Module

#### Check data

In [3]:
importlib.reload(mil_configs)
mil_tiles = mil_configs.SubPandaImageNetMIL(is_finetune=True)
mil_tiles.setup()

In [4]:
data_module = mil_tiles.get_data_module()
tiles_dataloader = data_module.train_dataloader()

Loading dataset from /tmp/innereye_cache1/SubPandaImageNetMIL-ImageNetEncoder/train_dataset.pt into cpu


In [5]:
tiles_sample = next(iter(tiles_dataloader))
tiles_sample.keys()

dict_keys(['tile_id', 'slide_id', 'mask', 'tile_x', 'tile_y', 'occupancy', 'data_provider', 'slide_isup_grade', 'slide_gleason_score', 'image', 'image_path'])

In [13]:
len(tiles_sample["image_path"])

16

In [6]:
len(tiles_sample["slide_id"][0])

75

In [6]:
len(tiles_sample["image"])

16

In [6]:
tiles_sample["image"][2].shape

torch.Size([77, 3, 224, 224])

In [8]:
tiles_sample["slide_isup_grade"]

[tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]),
 tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3]),
 tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

#### Check model

In [7]:
tiles_model = mil_tiles.create_model()

/home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [67]:
out1, out2 = tiles_model(tiles_sample["image"][2])
out1.shape, out2.shape

(torch.Size([1, 6]), torch.Size([1, 77]))

In [49]:
tiles_model.label_column

<SlideKey.LABEL: 'label'>

In [50]:
tiles_sample["image"][2].dtype

torch.float32

In [81]:
tiles_sample["image"][2]

tensor([[[[1.0000, 1.0000, 1.0000,  ..., 0.8196, 0.8118, 0.8118],
          [1.0000, 1.0000, 1.0000,  ..., 0.8078, 0.7961, 0.7922],
          [1.0000, 1.0000, 1.0000,  ..., 0.8235, 0.8039, 0.7961],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.7020, 0.6157, 0.7098],
          [1.0000, 1.0000, 1.0000,  ..., 0.3451, 0.5137, 0.5686],
          [1.0000, 1.0000, 1.0000,  ..., 0.4039, 0.3961, 0.4824]],

         [[1.0000, 1.0000, 1.0000,  ..., 0.4824, 0.4863, 0.4824],
          [1.0000, 1.0000, 1.0000,  ..., 0.4784, 0.4784, 0.4627],
          [1.0000, 1.0000, 1.0000,  ..., 0.4588, 0.5176, 0.4431],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.4431, 0.2902, 0.4784],
          [1.0000, 1.0000, 1.0000,  ..., 0.1059, 0.2078, 0.1922],
          [1.0000, 1.0000, 1.0000,  ..., 0.2039, 0.2078, 0.1843]],

         [[1.0000, 1.0000, 1.0000,  ..., 0.7255, 0.7294, 0.7451],
          [1.0000, 1.0000, 1.0000,  ..., 0.7490, 0.7294, 0.7255],
          [1.0000, 1.0000, 1.0000,  ..., 0

## Whole Slide Loading test

#### Check data

In [8]:
importlib.reload(mil_configs)
mil_slides = mil_configs.SubSlidesPandaImageNetMIL()
mil_slides.setup()

In [13]:
mil_slides.is_finetune

True

In [9]:
data_module = mil_slides.get_data_module()
slides_dataloader = data_module.train_dataloader()

In [10]:
slides_sample = next(iter(slides_dataloader))
slides_sample.keys()

[Plugin: cucim.kit.cuslide] Loading the dynamic library from: /home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages/cucim/clara/cucim.kit.cuslide@22.02.00.so
[Plugin: cucim.kit.cuslide] Loading the dynamic library from: /home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages/cucim/clara/cucim.kit.cuslide@22.02.00.so
[Plugin: cucim.kit.cuslide] Loading the dynamic library from: /home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages/cucim/clara/cucim.kit.cuslide@22.02.00.so
[Plugin: cucim.kit.cuslide] Loading the dynamic library from: /home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages/cucim/clara/cucim.kit.cuslide@22.02.00.so
[Plugin: cucim.kit.cuslide] Loading the dynamic library from: /home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages/cucim/clara/cucim.kit.cuslide@22.02.00.so
Initializing plugin: cucim.kit.cuslide (interfaces: [cucim::io::IImageFormat v0.1]) (impl: cucim.kit.cuslide)
Initializing plugin:

dict_keys([<SlideKey.SLIDE_ID: 'slide_id'>, <SlideKey.IMAGE: 'image'>, <SlideKey.IMAGE_PATH: 'image_path'>, <SlideKey.MASK: 'mask'>, <SlideKey.MASK_PATH: 'mask_path'>, <SlideKey.LABEL: 'label'>, <SlideKey.METADATA: 'metadata'>])

In [11]:
slides_sample["image"].shape

torch.Size([2, 60, 3, 224, 224])

In [12]:
slides_sample["metadata"]

{'data_provider': ['karolinska', 'karolinska'],
 'isup_grade': tensor([0, 2]),
 'gleason_score': ['0+0', '3+4']}

In [9]:
slides_sample["label"]

tensor([5])

In [10]:
data_module.train_dataset.LABEL_COLUMN

'isup_grade'

### check model

In [6]:
slides_model = mil_slides.create_model()

/home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [12]:
slides_model.label_column

<SlideKey.LABEL: 'label'>

In [13]:
encoder = slides_model.encoder

In [9]:
out1, out2 = slides_model(slides_sample["image"][0])
out1.shape, out2.shape

(torch.Size([1, 6]), torch.Size([1, 60]))

In [10]:
slides_sample["image"][0]

tensor([[[[0.4549, 0.3686, 0.3961,  ..., 0.9451, 0.7373, 0.7451],
          [0.4588, 0.3804, 0.4118,  ..., 0.8392, 0.6941, 0.8118],
          [0.4549, 0.3647, 0.4196,  ..., 0.7608, 0.6667, 0.8784],
          ...,
          [0.8980, 0.8863, 0.8549,  ..., 0.8980, 0.8667, 0.8549],
          [0.8902, 0.8824, 0.8510,  ..., 0.9255, 0.8941, 0.8784],
          [0.8784, 0.8706, 0.8431,  ..., 0.9686, 0.9608, 0.9608]],

         [[0.2627, 0.1961, 0.1922,  ..., 0.4902, 0.3294, 0.2941],
          [0.2667, 0.2118, 0.2314,  ..., 0.4314, 0.3020, 0.3529],
          [0.2471, 0.1922, 0.2392,  ..., 0.3608, 0.2706, 0.3843],
          ...,
          [0.3725, 0.3569, 0.3451,  ..., 0.7373, 0.6941, 0.7294],
          [0.3686, 0.3529, 0.3451,  ..., 0.9216, 0.8314, 0.7725],
          [0.3608, 0.3490, 0.3412,  ..., 0.9843, 0.9137, 0.8078]],

         [[0.5922, 0.4706, 0.4745,  ..., 0.7137, 0.6078, 0.5765],
          [0.5686, 0.4941, 0.5137,  ..., 0.6745, 0.5608, 0.5922],
          [0.5333, 0.4941, 0.5294,  ..., 0

In [7]:
train_results = slides_model.training_step(slides_sample, 0)

../../../../src/histopathology/models/deepmil.py:205: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predicted_probs = self.activation_fn(bag_logits)
/home/t-kbouzid/miniconda3/envs/HimlHisto/lib/python3.7/site-packages/pytorch_lightning/core/lightning.py:416: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  "You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet."


AssertionError: No trainer is set for this module.